In [121]:
import pandas as pd 
import pickle
import matplotlib.pyplot as plt
import uproot
import numpy as np
from machine_learning_hep.selectionutils import selectfidacc
from machine_learning_hep.bitwise import filter_bit_df, tag_bit_df

var_all = ["run_number", "ev_id", "cand_type", "inv_mass", "pt_cand","y_cand", "pt_prong0","pt_prong1",
           "eta_prong0","eta_prong1"]

var_evt = ["is_ev_rej", "run_number", "ev_id", "centrality", "z_vtx_reco"]

# treeevtorig = uproot.open("/home/ginnocen/alice/AliPhysics/PWGHF/treeHF/macros/validationtools/AnalysisResults.root")["PWGHF_TreeCreator/tree_event_char"]
# treeereco = uproot.open("/home/ginnocen/alice/AliPhysics/PWGHF/treeHF/macros/validationtools/AnalysisResults.root")["PWGHF_TreeCreator/tree_LctopKpi"]
treeevtorig = uproot.open("/home/ginnocen/alice/AliPhysics/PWGHF/treeHF/macros/validationtools/AnalysisResults.root")["PWGHF_TreeCreator/tree_event_char"]
treeereco = uproot.open("/home/ginnocen/alice/AliPhysics/PWGHF/treeHF/macros/validationtools/AnalysisResults.root")["PWGHF_TreeCreator/tree_D0"]
dfevtorig = treeevtorig.pandas.df(branches=var_evt)
dfreco = treeereco.pandas.df(branches=var_all)
dfreco = pd.merge(dfreco, dfevtorig, on=["run_number","ev_id"])
dfreco = dfreco.query("pt_cand>1 & pt_cand<9999")
isselacc = selectfidacc(dfreco.pt_cand.values, dfreco.y_cand.values)
dfreco = dfreco[np.array(isselacc, dtype=bool)]
dfreco["selS"] = np.array(tag_bit_df(dfreco, "cand_type", [[0],[]]), dtype=int)
dfreco = dfreco.query("selS==1")
dfreco = dfreco.query("is_ev_rej==0")
print(dfreco.inv_mass)

KeyError: not found: b'tree_event_char'
 in file: /home/ginnocen/alice/AliPhysics/PWGHF/treeHF/macros/validationtools/AnalysisResults.root

In [122]:
from ROOT import TFile, TH1F, TCanvas
from root_numpy import fill_hist

In [117]:
h_invmass = TH1F("hmass", "hmass",100, 1.6, 2.2)
fill_hist(h_invmass, dfreco.inv_mass)

Warning in <TFile::Append>: Replacing existing TH1: hmass (Potential memory leak).


In [118]:
cYields = TCanvas('cYields', 'The Fit Canvas')
h_invmass.Draw()
cYields.SaveAs("test.pdf")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cYields
Info in <TCanvas::Print>: pdf file test.pdf has been created
